<center>

<font size="5">MGMTMSA 408: Operations Analytics</font>

</center>


<center>

<font size="5">Assignment 2: Constraint Generation and Assortment Optimization</font>

</center>


<div style="display: flex; justify-content: space-between;">
    <div style="text-align: left;">
        Due on BruinLearn by 1:00pm on May 06, 2024.
    </div>
    <div style="text-align: right;">
        Arnav Garg (906310841) <br>
    </div>
</div>


**<font size="5">1. Learning Coffee Preferences</font>**

In this problem, we will be using column / constraint generation to learn about customer preferences
through a discrete choice model. In a discrete choice model, there are n distinct products, indexed
from 1 to n, as well as a dummy product 0, which represents the outside or no-purchase option
(i.e., the customer chooses not to buy any of the products from the assortment). When a customer
makes a decision, they choose a product from within the assortment or they choose the no-purchase
option. The specific type of discrete choice model we will use is the ranking-based choice model,
which will be explained in more detail in Part 1 and Part 2 of this problem.
The data set that we have consists of different assortments of coffee brands, which are listed
below. For each assortment, we observe how many customers purchased each of those brands, as
well as how many customers did not purchase anything when they considered the assortment. The
data will be described in more detail in Part 3.

Product # | Brand|
--- | --- |
1 | Peets |
2 | Folgers |
3 | Starbucks |
4 | Philz |
5 | Blue Bottle |
6 | Intelligentsia |
7 | Counter Culture |
8 | Stumptown |
9 | Groundworks |

After introducing the data in Part 3, we will focus on estimating the ranking-based model from
data by solving a linear program. Part 4 introduces this linear program and walks through solving
it with a fixed set of rankings. Part 5 then involves applying the column randomization approach,
while Part 6 involves using the column generation / constraint generation approach to solving the
problem. Finally, Part 7 looks at understanding the ranking model obtained from Part 6. A Jupyter
Notebook containing code snippets and skeleton code, HW2 - Coffee Code.ipynb, is provided to
help you with this problem.

**<font size="4">Part 1: Modeling preferences through rankings</font>**

In the ranking-based model, we assume that the customer population consists of K different cus-
tomer types. We let k denote the index of the customer type, which ranges from 1 to K.
Each customer type k chooses according to a ranking σk of the products.
As an example, suppose that n = 9. Suppose that we have three customer types, which are
given by the rankings

where the notation i ≺ j means that we prefer option i to option j.
The first ranking σ1 represents customers who prefer product 4 the most, followed by product
3, followed by product 0 (the no-purchase option), then followed by the remaining products. The
second ranking σ2 represents customers who prefer product 3 the most, followed by product 2,
followed by product 4, followed by the no-purchase option, then followed by the remaining products.
The third ranking σ3 represents customers who prefer product 2 the most, followed by product 5,
followed by the no-purchase option, followed by all of the other products.
Each customer type k also has a probability λk associated with it; we can think of λk as the
probability that a randomly drawn customer will be of type k, or equivalently, what fraction of
customers in the population are of type k (i.e., they will choose according to the ranking σk).
Suppose that the probabilities of the three rankings are λ1 = 0.1, λ2 = 0.3, λ3 = 0.6. Then the
choice probabilities of the products are

In this example, customer types 1 and 2 both choose product 4, because it is their most preferred
option out of products 1, 4 and 5 and the no-purchase option 0. As a result, the probability that a
random customer chooses product 4 is 0.4. Customer type 3 chooses product 5, because it is their
most preferred option. Therefore, the probability that a random customer chooses product 5 is 0.6.
None of the three customer types choose either product 1 or the no-purchase option, and hence
their choice probabilities are 0.

a) For the same rankings given in the example, calculate the choice probabilities for the set of
products {3, 4}.

\begin{align}
\text{maximize} \;\; & 500x_1 + 800x_2 + 1000x_3 \;\;\;\; \text{[Objective Function]}\\
\text{s.t.} \;\; & 14x_1 + 20x_2 + 40x_3 \le 300 \;\;\;\; \text{[Germanium Constraint]}\\
            & 30x_1 + 20x_2 + 15x_3 \le 200 \;\;\;\; \text{[Silicon Constraint]}\\
            & 20x_1 + 30x_2 + 50x_3 \le 1080 \;\;\;\; \text{[Time Constraint]}\\
            & x_i \ge 0 \quad i = 1 \ldots 3 \;\;\;\; \text{[Non-Negativity Constraint]}\\
\end{align}


where decision variables are given as follows:

$x_1$ is the number of chips produced of type 1

$x_2$ is the number of chips produced of type 1

$x_3$ is the number of chips produced of type 1

to maximize revenue

b) For the same rankings given in the example, calculate the choice probabilities for the set of
products {2, 3, 4}.

In [17]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *
import numpy as np
import pandas as pd

##############################################################################################
# Initialize the model 
primal = Model()

# Set the OutputFlag parameter to 0 to disable logging
primal.Params.outputflag = 0

##############################################################################################
# Define Variables
x1 = primal.addVar(lb = 0)
x2 = primal.addVar(lb = 0)
x3 = primal.addVar(lb = 0)

##############################################################################################
# Define Constraints
## Germanium Transistors
Ge_constr = primal.addConstr(14 * x1 + 20 * x2 + 40 * x3 <= 300)

## Silicon Transistors
Si_constr = primal.addConstr(30 * x1 + 20 * x2 + 15 * x3 <= 200)

## Time Constraint
t_constr = primal.addConstr(20 * x1 + 30 * x2 + 50 * x3 <= 1080)

##############################################################################################
# Construct Objective
primal.setObjective(500 * x1 + 800 * x2 + 1000 * x3, GRB.MAXIMIZE)

##############################################################################################
# Update and solve
primal.update()
primal.optimize()

##############################################################################################
# Print Optimal Solution
print("Maximum Profit(Optimal Objective Value): $", primal.objVal)

##############################################################################################

Maximum Profit(Optimal Objective Value): $ 9600.0


**<font size="4">Part 2: Choice probabilities as a linear system of equations</font>**

a) For the M = 3 assortments in Part 1 (S1 = {1, 4, 5}, S2 = {3, 4}, S3 = {2, 3, 4}), and for the
K = 3 customer types (rankings) given in Part 1, write down the A matrix. Your matrix should
have 30 rows and 3 columns.
Hint: if you are having difficulty with this problem, take a look at the function permToA provided
in the skeleton code.

\begin{align}
\text{minimize} \;\; & 300w_{Ge} + 200w_{Si} + 1080w_t \;\;\;\; \text{[Objective Function]} \\
\text{s.t.} \;\; & 14w_{Ge} + 30w_{Si} + 20w_t \ge 500 \;\;\;\; \text{[Weight of ChipType1 Constraint]} \\
            & 20w_{Ge} + 20w_{Si} + 30w_t \ge 800 \;\;\;\; \text{[Weight of ChipType2 Constraint]} \\
            & 40w_{Ge} + 15w_{Si} + 50w_t \ge 1000 \;\;\;\; \text{[Weight of ChipType3 Constraint]} \\
            & w_{Ge}, w_{Si}, w_t \ge 0 \;\;\;\; \text{[Non-Negativity Constraint]}\\
\end{align}


where decision variables are given as follows:

$w_{Ge}$ is the weight of the Germanium constraint

$w_{Si}$ is the weight of the Germanium constraint

$w_{t}$ is the weight of the Germanium constraint

to get the lowest bound on revenue

b) Using this matrix, compute v = Aλ, where λ = (0.1, 0.3, 0.6). What do you get for v?

In [19]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *
import numpy as np
import pandas as pd

##############################################################################################
# Initialize the model 
dual = Model()

# Set the OutputFlag parameter to 0 to disable logging
dual.Params.outputflag = 0

##############################################################################################
# Define Variables
w_Ge = dual.addVar(lb = 0)
w_Si = dual.addVar(lb = 0)
w_t = dual.addVar(lb = 0)

##############################################################################################
# Define Constraints
## Weight of chip type 1 constraint
chip1_constr = dual.addConstr(14 * w_Ge + 30 * w_Si + 20 * w_t >= 500)

## Weight of chip type 2 constraint
chip2_constr = dual.addConstr(20 * w_Ge + 20 * w_Si + 30 * w_t >= 800)

## Weight of chip type 3 constraint
chip3_constr = dual.addConstr(40 * w_Ge + 15 * w_Si + 50 * w_t >= 1000)

##############################################################################################
# Construct Objective
dual.setObjective(300 * w_Ge + 200 * w_Si + 1080 * w_t, GRB.MINIMIZE)

##############################################################################################
# Update and solve
dual.update()
dual.optimize()

##############################################################################################
# Print Optimal Solution
print("Lowest Bound on Profit(Optimal Objective Value): $", dual.objVal)

##############################################################################################

Lowest Bound on Profit(Optimal Objective Value): $ 9600.0


**<font size="4">Part 3: Understanding the data</font>**

a) Which assortment has the most transactions?

b) Which assortment has the least transactions?

c) How many assortments include the Starbucks brand? Convert the matrix in coffee transaction counts.csv into an array of choice probabilities,
i.e., divide each row by its row sum. If you performed this step correctly, then the choice probability
of the no-purchase option for the first assortment should be 0.450413.

d) For assortment m = 10, what fraction of customers purchased the Starbucks brand?

e) What fraction of customers offered assortment m = 3 chose to not purchase anything?

**<font size="4">Part 4: Estimating the ranking-based model</font>**

**<font size="5">2. Cloud Computing</font>**

Cirrus is a cloud computing provider that offers its users the ability to reserve computational
capacity on its cloud. In this problem, we will analyze how Cirrus should allocate its capacity.
Cirrus sells its computational resources in units called instances. These are virtual computers
that a user can reserve for a whole day. Each instance consist of three components:

• Central processing units (CPUs): each instance reserves some number of CPUs, range from 1
CPU to 64 CPUs. \
• Memory: each instance reserves some amount of memory, ranging from 1 GB to 128GB, in
increments of 1 GB. \
• Graphical processing units (GPUs): each instance reserves some amount of GPU memory,
ranging from 1 GB to 8 GB.

In total, on a given day, Cirrus has 512 CPUs, 1024 GB of memory and 64 GB of GPU memory
available.
Based on its user requirements, offers the following instances:

Instance | Name | CPU(#) | Memory(GB) | GPU(GB) | Price | Rate (#/day)
--- | --- | --- | --- | --- | --- | ---
1 | C1 | 16 | 8 | 1 | $7 | 5.0
2 | C2 | 32 | 16 | 1 | $12 | 5.0
3 | C3 | 64 | 32 | 1 | $24 | 1.8
4 | M1 | 8 | 32 | 1 | $22 | 3.0
5 | M2 | 16 | 64 | 1 | $44 | 2.6
6 | M3 | 32 | 128 | 1 | $88 | 1.0
7 | G1 | 16 | 16 | 2 | $30 | 0.8
8 | G2 | 32 | 32 | 6 | $90 | 0.4
9 | G3 | 64 | 64 | 8 | $120 | 0.3

Columns 3 - 5 specify the hardware requirements of each instance type. Column 6 specifies the
price for activating the instance. We will also assume that an instance is reserved for a whole day.

Column 7 specifies the arrival rate. This is the number of users requesting an instance of this
type each day. In this problem, we’ll assume that the demand for each type of instance follows
a Poisson arrival process, with the arrival rate as given in the table above. We will assume that
Cirrus only allows reservations for a period of 5 days. We will also assume that this period of 5 days
precedes the day for which the instance is reserved and is exclusive of this day (i.e., the requests
are not fulfilled during the reservation period). For example, instance requests can come in on
Monday, Tuesday, Wednesday, Thursday and Friday, at any time on those days; the instances that
are accepted are then reserved for the whole day of Saturday. Instances are not reserved during the
5 day reservation period.

Cirrus is interested in understanding how it should allocate its limited capacity to the different
types of instances. Cirrus currently accepts requests in a first-come first-serve fashion. In this
problem, we will approach Cirrus’s problem from a revenue management lens.

**<font size="4">Part 1: Capacity control formulationn</font>**

In this first part of the problem, we need to mathematically formulate the capacity control problem
for Cirrus as a linear optimization problem. Let x1, . . . , x9 be the number of instances that are
reserved of each instance type. These will be the decision variables of our problem.

a) Based on the prices given in the table earlier, what is the objective function of the problem?

\begin{align}
\text{maximize} \;\; & 7x_1 + 12x_2 + 24x_3 + 22x_4 + 44x_5 + 88x_6 + 30x_7 + 90x_8 + 120x_9 \\
\end{align}


b) In terms of x1, . . . , x9, what is the constraint on the total memory usage of the instances that
are reserved?

\begin{align}
\text{s.t.} \;\; & 8x_1 + 16x_2 + 32x_3 + 32x_4 + 64x_5 + 128x_6 + 16x_7 + 32x_8 + 64x_9 \le 1024\\
\end{align}


c) Let’s now consider the forecasted demand for each instance type. Recall that for a Poisson
arrival process with rate λ per unit time, the total number of arrivals in a period of length T is
a Poisson random variable Y with mean λT .
Over the five day period, what is the expected number of requests of instance type 5? Assuming
there will be exactly this many requests over the T = 5 day period, what is the constraint on
the number of requests of instance type 5 we may accept?

Over the five day period, the expected number of requests of instance type 5 is 13. The constraint on the number of requests of instance type 5 we may accept is given as follows:

$\text{Expected Number of Requests} = \lambda_5T = \text{Rate} \times \text{Time} = 2.6 \times 5 = 13$

$\text{s.t.} \;\; x_5 \le 13 \;\;\;\; \text{[Constraint]}$

d) Using the logic in (a) – (c), write down the mathematical formulation of the T = 5 day capacity
control problem as a linear optimization problem.

\begin{align}
\text{maximize} \;\; & 7x_1 + 12x_2 + 24x_3 + 22x_4 + 44x_5 + 88x_6 + 30x_7 + 90x_8 + 120x_9 \\
\text{s.t.} \;\; & 16x_1 + 32x_2 + 64x_3 + 8x_4 + 16x_5 + 32x_6 + 16x_7 + 32x_8 + 64x_9 \le 512 \\
& 8x_1 + 16x_2 + 32x_3 + 32x_4 + 64x_5 + 128x_6 + 16x_7 + 32x_8 + 64x_9 \le 1024 \\
& 1x_1 + 1x_2 + 1x_3 + 1x_4 + 1x_5 + 1x_6 + 2x_7 + 6x_8 + 8x_9 \le 64 \\
& x_1 \ge 0, x_1 \le 25 \\
& x_2 \ge 0, x_2 \le 25 \\
& x_3 \ge 0, x_3 \le 9 \\
& x_4 \ge 0, x_4 \le 15 \\
& x_5 \ge 0, x_5 \le 13 \\
& x_6 \ge 0, x_6 \le 5 \\
& x_7 \ge 0, x_7 \le 4 \\
& x_8 \ge 0, x_8 \le 2 \\
& x_9 \ge 0, x_9 \le 1.5
\end{align}


**<font size="4">Part 2: Solving the capacity control problem in Python/Gurobi</font>**

Now, implement your formulation in Part 1(d) using Python and Gurobi.

a) Solve the problem. What is the optimal objective value?

In [25]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *
import numpy as np
import pandas as pd

##############################################################################################
# Initialize the model 
m = Model()

# Set the OutputFlag parameter to 0 to disable logging
m.Params.outputflag = 0

##############################################################################################
# Define Variables
x = m.addVars(9, lb = 0)

##############################################################################################
# Define Constraints
## CPU Constraint
cpu = np.array([16, 32, 64, 8, 16, 32, 16, 32, 64])
CPU_constr = m.addConstr(sum(cpu[i]*x[i] for i in range(len(cpu))) <= 512)

## Memory Constraint
memory = np.array([8, 16, 32, 32, 64, 128, 16, 32, 64])
Mem_constr = m.addConstr(sum(memory[i]*x[i] for i in range(len(memory))) <= 1024)

## GPU Constraint
gpu = np.array([1, 1, 1, 1, 1, 1, 2, 6, 8])
GPU_constr = m.addConstr(sum(gpu[i]*x[i] for i in range(len(gpu))) <= 64)

## Rate Constraint
rate = np.array([25, 25, 9, 15, 13, 5, 4, 2, 1.5])
for i in range(len(rate)):
    m.addConstr(x[i] <= rate[i])

##############################################################################################
# Construct Objective
revenue = np.array([7, 12, 24, 22, 44, 88, 30, 90, 120])
m.setObjective(sum(revenue[i]*x[i] for i in range(len(revenue))), GRB.MAXIMIZE)

##############################################################################################
# Update and solve
m.update()
m.optimize()

##############################################################################################
# Print Optimal Solution
print("Maximum Revenue(Optimal Objective Value): $", m.objVal)

##############################################################################################

Maximum Revenue(Optimal Objective Value): $ 1039.4285714285716


b) In the optimal allocation, how many requests of instance type C1 are accepted?

In [26]:
print("Optimal Allocation:")
for i in range(len(revenue)):
    print("Instance Type", i+1, ":", x[i].x)

Optimal Allocation:
Instance Type 1 : 6.285714285714286
Instance Type 2 : 0.0
Instance Type 3 : 0.0
Instance Type 4 : 3.428571428571429
Instance Type 5 : 0.0
Instance Type 6 : 5.0
Instance Type 7 : 4.0
Instance Type 8 : 2.0
Instance Type 9 : 1.5


We observe that in the optimal allocation, ~6.29 requests of instance type C1 are accepted.

**<font size="4">Part 3: Simulating current practice</font>**

Cirrus would like to understand how well its current policy does. Currently, Cirrus simply accepts
requests in a first-come first-serve fashion, without considering the revenue of the requests. In this
part of the problem, we will simulate Cirrus’s current policy.

Set your random seed to 10 using the random.seed() function in numpy. Using the provided
function generateArrivalSequences in the HW1 - Cloud Code.ipynb notebook, generate 100 se-
quences of request arrivals, using the rates provided in the table above, over a period of T = 5 days.
This function will generate an array of arrays:

• arrival sequence times: This contains the time at which each request arrives, in the interval
[0, 5]. For example, arrival sequence times[0][1] is the time at which the second request
arrives in the first sequence. \
• arrival sequence types: The instance type of each request. (Note: the instance types in
this array are numbered from 0 to 8, in accordance with how Python’s numbering starts at
zero. Thus, for example, a value of 5 in this array indicates instance type 6 / M3.)

If you have done this step correctly, the first three times in arrival sequences times[0] should
be 0.07414243 0.1246028 0.15928449.

In [27]:
# Preconditions:
# nSimulations = integer specifying number of simulations to run
# rates = array containing arrival rate (# / day) for each of the instance
# types (should be an array with 9 elements)
# T = length of horizon in days.

def generateArrivalSequences( nSimulations, rates, T ):
    total_rate = sum(rates)
    nTypes = len(rates)

    arrival_sequences_times = []
    arrival_sequences_types = [];

    for s in range(nSimulations):
        single_arrival_sequence_time = [];
        single_arrival_sequence_type = [];
        t = 0;
        while (t < T):
            single_time = np.random.exponential(1.0/total_rate)
            single_type = np.random.choice(nTypes, p = rates/total_rate)

            t += single_time;

            if (t < T):
                single_arrival_sequence_time.append(t)
                single_arrival_sequence_type.append(single_type)
            else:
                break

        arrival_sequences_times.append(np.array(single_arrival_sequence_time))
        arrival_sequences_types.append(np.array(single_arrival_sequence_type))
    return arrival_sequences_times, arrival_sequences_types



# Code to test out above function
np.random.seed(1)
nSimulations_test = 100
rates_test = np.array([5.0, 2.0, 3.0])
T_test = 8
times, types = generateArrivalSequences(nSimulations_test, rates_test, T_test)

# If code above is working correctly, code below should show
# value of 80.71:
counts = np.array([len(types[i]) for i in range(nSimulations_test)] )
counts.mean()

80.71

In [28]:
np.random.seed(10)
nSimulations = 100
rates = np.array([5, 5, 1.8, 3, 2.6, 1, 0.8, 0.4, 0.3])
T = 5
times, types = generateArrivalSequences(nSimulations, rates, T)
times[0]

array([0.07414243, 0.1246028 , 0.15928449, 0.1703762 , 0.17968569,
       0.23779251, 0.23799131, 0.32214315, 0.38642729, 0.51196926,
       0.55126954, 0.57475458, 0.60405595, 0.65238384, 0.70519578,
       0.78740185, 0.90765536, 0.91241996, 0.9185014 , 0.92091505,
       0.96077223, 0.97191941, 0.99369472, 1.01132902, 1.03111801,
       1.05616581, 1.14264621, 1.16700269, 1.38771201, 1.47562187,
       1.52133751, 1.55976805, 1.56178183, 1.56595073, 1.58612921,
       1.58817843, 1.60718495, 1.62855072, 1.73506112, 1.84081623,
       1.88756662, 1.93334033, 1.95132628, 1.96946998, 2.01044946,
       2.04284187, 2.04617896, 2.06703689, 2.25677443, 2.27602698,
       2.3193975 , 2.32294156, 2.39947557, 2.44379428, 2.46549244,
       2.48339641, 2.53963883, 2.54224165, 2.54798842, 2.65703572,
       2.66944199, 2.67512209, 2.70700517, 2.71503145, 2.76708375,
       2.78144725, 2.79437102, 2.92271451, 2.96474668, 2.96549902,
       3.0082648 , 3.04969828, 3.09379581, 3.10187596, 3.16136

a) What is the average number of arrivals of type C1 in the set of simulated sequences?

In [29]:
f = 0
for i in range(nSimulations):
    f += sum(types[i] == 0) #c1 is 0
print("Average number of arrivals of type C1 =", f/nSimulations)

Average number of arrivals of type C1 = 26.63


b) What is the average number of arrivals, of all types, over the set of simulated sequences? Does
this make sense? (Hint: What is the expected value of the sum of Poisson random variables?)

In [30]:
instance_types = ['C1', 'C2', 'C3', 'M1', 'M2', 'M3', 'G1', 'G2', 'G3']
avg_arrivals = np.zeros(len(instance_types))

for j in range(len(instance_types)):
    for i in range(nSimulations):
        avg_arrivals[j] += sum(types[i] == j)
    print("Average number of arrivals of type", instance_types[j], "=", avg_arrivals[j]/nSimulations)
print("Average number of arrivals of all types =", sum(avg_arrivals)/nSimulations)

Average number of arrivals of type C1 = 26.63
Average number of arrivals of type C2 = 24.38
Average number of arrivals of type C3 = 9.01
Average number of arrivals of type M1 = 14.88
Average number of arrivals of type M2 = 13.32
Average number of arrivals of type M3 = 4.93
Average number of arrivals of type G1 = 4.2
Average number of arrivals of type G2 = 2.27
Average number of arrivals of type G3 = 1.38
Average number of arrivals of all types = 101.0


The average number of arrivals, of all types, over the set of simulated sequences is 101.
The expected value of the sum of Poisson random variables is the sum of the expected values of the individual random variables. The expected value of a Poisson random variable is its rate parameter, so the expected value of the sum of Poisson random variables is the sum of individual rate parameters which comes out to be 19.9 in this case. Hence, over a period of 5 days, the expected value for average number of arrivals of all types is 99.5 which is approximately what we get.

$\lambda_{all} = \sum_{i=0}^{9}\lambda_i = 19.9$ \
$\text{Expected Value} = \lambda_{all} * T = 19.9*5 = 99.5$

c) Next, implement Cirrus’s current policy. This policy accepts any request, so long as there is
capacity for it. You may use the code skeleton given in HW1 - Cloud Code.ipynb as a starting
point.
What is the average revenue garnered by this policy over the 100 simulated sequences?

In [31]:
# Preconditions for code below:
nSimulations = 100
nResources = 3
B = np.array([512, 1024, 64])
arrival_sequences_times = times
arrival_sequences_types = types

results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
    b = B.copy()
    single_revenue = 0.0 # will contain the revenue of this simulation
    nArrivals = len(arrival_sequences_times[s])

    # Go through the arrivals in sequence
    for j in range(nArrivals):
        # Obtain the time of the arrival, and its type (i)
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Check if there is sufficient capacity for the request
        if (b[0] >= cpu[i]) and (b[1] >= memory[i]) and (b[2] >= gpu[i]):
            # If there is sufficient capacity, accrue the revenue
            single_revenue += revenue[i]
            # and remove the cpu capacity
            b[0] -= cpu[i]
            # and remove the memory capacity
            b[1] -= memory[i]
            # and remove the gpu capacity
            b[2] -= gpu[i]

    # Save the results of this simulation here
    results_myopic_revenue[s] = single_revenue
    results_myopic_remaining_capacity[:, s] = b

# Find the average revenue
revenue_myopic = results_myopic_revenue.mean()

# Find the average remaining quantity of each resource
remaining_cpu_myopic = results_myopic_remaining_capacity[0, :].mean()
remaining_memory_myopic = results_myopic_remaining_capacity[1, :].mean()
remaining_gpu_myopic = results_myopic_remaining_capacity[2, :].mean()

# Print results
print("Average Revenue: $", revenue_myopic)

Average Revenue: $ 528.28


d) What is the average remaining capacity (of CPUs, memory and GPUs) of this policy?

In [32]:
# Print results
print("Average Remaining CPU Capacity: ", remaining_cpu_myopic)
print("Average Remaining Memory Capacity: ", remaining_memory_myopic)
print("Average Remaining GPU Capacity: ", remaining_gpu_myopic)

Average Remaining CPU Capacity:  0.24
Average Remaining Memory Capacity:  371.52
Average Remaining GPU Capacity:  37.42


**<font size="4">Part 4: A bid-price control policy</font>**

Let’s now develop a bid-price control policy, using the solution of the LP.

a) As a warm-up, suppose that we receive a request for instance type 5 (M2). Let π1, π2, π3 be the
shadow prices / dual values of the constraints for CPUs, memory and GPUs at a particular point
in time. In terms of the shadow prices, what is the approximate opportunity cost of accepting
this request?

Opportunity Cost = $16\pi_1 + 64\pi_2 + 1\pi_3$

b) As a further warm-up, suppose that we receive a request at time t. What is the expected number
of requests of type i we will receive from time t to time T (including both t and T )?

Expected # of Requests = $\lambda_i(T-t+1)$

c) Next, implement Cirrus’s current policy. This policy accepts any request, so long as there is
capacity for it. You may use the code skeleton given in HW1 - Cloud Code.ipynb as a starting
point.
What is the average revenue garnered by this policy over the 100 simulated sequences?

In [33]:
# Preconditions for code below:
nSimulations = 100
nInstances = 9
nResources = 3
B = np.array([512, 1024, 64])
arrival_sequences_times = times
arrival_sequences_types = types

# As we did in-class, define a function bpc() to re-solve the LO each time:
def bpc(b, t):
    # for r in range(nResources):
    #     # Set the RHS of the resource constraint to b[r] here
    CPU_constr.rhs = b[0]
    Mem_constr.rhs = b[1]
    GPU_constr.rhs = b[2]

    for k in range(nInstances):
        # Set the RHS of the forecast constraint for each instance
        # type to the expected number of requests over the duration
        # of the remaining horizon (T - t).
        # ...
        x[k].ub = rates[k] * (5 - t)

    # Re-solve the model:
    m.update()
    m.optimize()

    # Obtain the dual values/shadow prices
    dual_val = cpu[g]*CPU_constr.pi + memory[g]*Mem_constr.pi + gpu[g]*GPU_constr.pi

    # Return the dual values:
    return dual_val


results_bpc_revenue = np.zeros(nSimulations)
results_bpc_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
    b = B.copy()
    single_revenue = 0.0 # will contain the revenue of this simulation
    nArrivals = len(arrival_sequences_times[s])

    # Go through the arrivals in sequence
    for j in range(nArrivals):
        # Obtain the time of the arrival, and its type (g)
        arrival_time = arrival_sequences_times[s][j]
        g = arrival_sequences_types[s][j]

        # Check if there is enough capacity
        if (b[0] >= cpu[g]) and (b[1] >= memory[g]) and (b[2] >= gpu[g]):
            # Re-solve the LO and obtain the dual values
            dual_val = bpc(b, arrival_time)

            # Check if the revenue is at least the sum of the bid prices:
            if (revenue[g] >= dual_val):
                # If there is sufficient capacity, accrue the revenue
                single_revenue += revenue[g]
                # and remove the cpu capacity
                b[0] -= cpu[g]
                # and remove the memory capacity
                b[1] -= memory[g]
                # and remove the gpu capacity
                b[2] -= gpu[g]

    # Save the results of this simulation here:
    results_bpc_revenue[s] = single_revenue
    results_bpc_remaining_capacity[:, s] = b


# Find the average revenue
revenue_bpc = results_bpc_revenue.mean()

# Find the average remaining quantity of each resource
remaining_cpu_bpc = results_bpc_remaining_capacity[0, :].mean()
remaining_memory_bpc = results_bpc_remaining_capacity[1, :].mean()
remaining_gpu_bpc = results_bpc_remaining_capacity[2, :].mean()

# Print results
print("Average Revenue: $", revenue_bpc)

Average Revenue: $ 925.59


d) What is the average remaining capacity (of CPUs, memory and GPUs) of this policy?

In [34]:
# Print results
print("Average Remaining CPU Capacity: ", remaining_cpu_bpc)
print("Average Remaining Memory Capacity: ", remaining_memory_bpc)
print("Average Remaining GPU Capacity: ", remaining_gpu_bpc)

Average Remaining CPU Capacity:  27.2
Average Remaining Memory Capacity:  4.88
Average Remaining GPU Capacity:  20.62
